In [ ]:
# In[1]:
import pandas as pd
import numpy as np
import os
import pydicom
import cv2

rootdir = ''


train_y_cala = pd.read_csv(rootdir + "calc_case_description_test_set.csv")
train_y_cala = [train_y_cala.assessment, train_y_cala.pathology]

## Standard dimension of the image
std_height = 299
std_width = 299
train_x_cala = np.zeros(shape = (len(train_y_cala[0]), std_height, std_width))
num_image_cala = 0
train_x_cala_ori_size = np.zeros(shape = (len(train_y_cala[0]), 2))
train_x_index = np.zeros(shape = (2 * len(train_y_cala[0])))
train_x_missing = []
train_x_total_num = 0
# rootdit: CBIS-DDSM is default  or other folder where you save images
def list_all_files_by_size(rootdir):
    global train_x_cala, num_image_cala, train_x_missing, train_x_total_num, train_x_index
    _files = []
    list = os.listdir(rootdir) # list all dir and file
    for i in range(0, len(list)):
        path = os.path.join(rootdir,list[i])
        if os.path.isdir(path):
            _files.extend(list_all_files_by_size(path))
        if os.path.isfile(path):
            # fsize = os.path.getsize(path)
            
            # read dcm image
            temp = pydicom.dcmread(path)
            if sum(sum(temp.pixel_array)) > 1000000:
                # resize and add new pixel array to train_x
                # interpolation is parameter of different methods to resize image
                train_x_cala_ori_size[num_image_cala] = [len(temp.pixel_array), len(temp.pixel_array[0])]
                train_x_cala[num_image_cala] = cv2.resize(temp.pixel_array, (std_width, std_height), interpolation = cv2.INTER_CUBIC)
                num_image_cala += 1
                train_x_index[train_x_total_num] = 1
                # continue
            
            _files.append(path)
            train_x_total_num += 1

    return _files

def find_missing(train_x_index, train_x_missing):
    for i in range(0, len(train_x_index)):
        if (i > 0) & (i % 2 == 1) & (train_x_index[i - 1] == 0) & (train_x_index[i] == 0):
            train_x_missing.append(int(i / 2))

def remove_index(image, _list):
    image_index = [i for i in range(len(image))]
    image_index = set(image_index)
    black_index = set(_list)
    index = list(image_index - black_index)
    new_image = [image[i] for i in index]
    return new_image

_list = list_all_files_by_size(rootdir + 'CBIS-DDSM calc test")

find_missing(train_x_index, train_x_missing)

train_y_cala = remove_index(train_y_cala, train_x_missing)

In [ ]:
## Renaming elements

train_x = train_x_cala
train_y = train_y_cala

In [ ]:
## Defining shape of train_x, right input for the Inception V3 model

train_x = train_x.reshape(len(train_x),299,299,1)
train_x = np.repeat(train_x[...,], 3, -1)

## Normalizing the pixel values to [0,1] and saving as an array
train_x = train_x/65535
train_x = np.asarray(train_x)

In [ ]:
## Defining the labels & preprocessing them

train_labels = np.asarray(train_y[1])


# Setting all values above 1 to 1
train_labels[train_labels == "BENIGN"] = 0
# Setting all values above 1 to 1
train_labels[train_labels == "BENIGN_WITHOUT_CALLBACK"] = 0

train_labels[train_labels == "MALIGNANT"] = 1

train_labels = train_labels.astype(int)

In [ ]:
## Saving the data

np.save('calctest_x.npy', train_x)
np.save('calctest_y.npy', train_labels)